# Creation of your own docker image

The docker image should be described in a special file, typically named `dockerfile`.

### `docker build` - build image

- the last mandatory argument sets the build directory - you should use `.` to set the current directory.
- `-t` - set the name and tag of the image;
- `-f` - set the dockerfile name if it is different from the default `dockerfile`.

In [6]:
%%bash
cd image_creation_files
docker build -t custom_ubuntu:test -f my_first_dockerfile .
docker images | grep custom_ubuntu
docker rmi custom_ubuntu:test

#1 [internal] load build definition from my_first_dockerfile
#1 transferring dockerfile: 784B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 DONE 0.0s

#4 [1/4] FROM docker.io/library/ubuntu:latest
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 30B done
#5 DONE 0.0s

#6 [3/4] COPY script.sh script.sh
#6 CACHED

#7 [2/4] WORKDIR /home/Dranikf
#7 CACHED

#8 [4/4] RUN apt-get update && apt-get install -y vim wget curl git
#8 CACHED

#9 exporting to image
#9 exporting layers done
#9 writing image sha256:7105d4774a9281753707c0823de5eb9a6aad09b4c8248fcb732f08b0899b5c88 done
#9 naming to docker.io/library/custom_ubuntu:test done
#9 DONE 0.0s


custom_ubuntu              test      7105d4774a92   6 minutes ago   252MB
Untagged: custom_ubuntu:test
Deleted: sha256:7105d4774a9281753707c0823de5eb9a6aad09b4c8248fcb732f08b0899b5c88


You should see a difference between building directory and dockerfile. The `dockerfile` is only taken from the building directory if it is not specified in the `-f` option. But the files used for the build will in any case come from the build directory.

In the following example I take the dockerfile from `/image_creation_files` but set the build directory to `/image_creation_files/test_folder`. The dockerfile is designed to copy `scipt.sh` from the building directory, but in named directories `script.sh` is different, so let's check which one is executed.

In [18]:
%%bash
cd image_creation_files
echo "=====tree of the image_creation_files====="
tree

echo
echo

echo "=====script from image_creation_files====="
bash script.sh
echo "=====script from test_folder====="
bash test_folder/script.sh

echo
echo

docker build -t test_ubuntu -f my_first_dockerfile test_folder &> /dev/null
docker run --name test_ubuntu --rm -itd test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh
docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====tree of the image_creation_files=====
.
├── dockerfile
├── my_first_dockerfile
├── script.sh
└── test_folder
    └── script.sh

1 directory, 4 files


=====script from image_creation_files=====
It's a script.sh from the images_creation_files
=====script from test_folder=====
It's a script from test_folder


=====message from container=====
It's a script from test_folder


As a result, even though each dockerfile is in the `image_creation_files` folder, the `script.sh` file is taken from the folder specified as the build folder.

Just to check the opposite option. I will use docker file from `test_folder` but set `.` as build directory. So the script result will be from `images_creation_folder`. To know that container was built from dockerfile from `test_folder`, dockerfile will add "hello" to the result of the executed script.

In [29]:
%%bash
cd image_creation_files

echo "=====script from image_creation_files====="
bash script.sh
echo "=====script from test_folder====="
bash test_folder/script.sh

echo
echo

docker build -t test_ubuntu -f test_folder/dockerfile . &> /dev/null
docker run --name test_ubuntu --rm -itd test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh
docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====script from image_creation_files=====
It's a script.sh from the images_creation_files
=====script from test_folder=====
It's a script from test_folder


=====message from container=====
It's a script.sh from the images_creation_files
hello


**Note** even if you have specified a folder as some assembly folder, you must still pass in the `-f` parameter the path relative to the startup folder.

The next two cells show the difference:

- Not specifying a folder for `my_first_dockerfile` will cause an error;

In [35]:
%%bash
docker image build -f my_first_dockerfile -t ubuntu_test image_creation_files &> /dev/null
docker rmi ubuntu_test &> /dev/null

CalledProcessError: Command 'b'docker image build -f my_first_dockerfile -t ubuntu_test image_creation_files &> /dev/null\nsleep 10\ndocker rmi ubuntu_test &> /dev/null\n'' returned non-zero exit status 1.

- Specifying a folder leads to successful completion.

In [36]:
%%bash
docker image build -f image_creation_files/my_first_dockerfile -t ubuntu_test image_creation_files &> /dev/null
docker rmi ubuntu_test &> /dev/null